# Trial 2

We use the basic framework from Q learning instead of what is outlined in the paper.

1. Import statements
    * The most important part here is the import for the function that outputs the values
    
2. Delcaring Parameters

3. Setting up the convolutional network

4. Training the network

This would be a Deep Q network to start off with. The problems I forsee with this is the insanely large layers which may impede performance and the modeling of the system. The accuracy and performance of this method, whislt simple, may not be up too par either. 

In [4]:
import numpy as np
import random
import tensorflow as tf
#import matplotlib.pyplot as plt
#^ Don't have this library yet, but its just for plotting
#also need to import simulator

In [6]:
tf.reset_default_graph()

size = 16
#size = sim.getsize() #simulator needs to have a function for getting size of problem

In [7]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,size],dtype=tf.float32) #The 16 in this line needs to get changed to however many
                                                        #elements there are to optimize
    
W = tf.Variable(tf.random_uniform([size,size],0,0.01))   #The 16 corresponds to the number of elements to optimize
                                                    #The 4 corresponds to the number of moves at each step, which 
                                                    #is equivalent to the number of elements in our case
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,size],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [9]:
init = tf.initialize_all_variables()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = sim.reset() #make this as well
        rAll = 0

        j = 0
        #The Q-Network
        while j < size:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(size)[s:s+1]}) #replaced
            if np.random.rand(1) < e:
                a[0] = sim.getRandomSample(); #Need to make this function
            #Get new state and reward from environment
            s1,r = sim.step(a[0]) #need to make this function, gets reward for doing action A and returns new state as well
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(size)[s1:s1+1]}) #replaced
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1) #change here?
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1 #change here also for min?
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(size)[s:s+1],nextQ:targetQ}) #Replaced
            rAll += r
            s = s1
           
        
        e = 1./((i/50) + 10) #reduce random chance as training progresses

        #keep the best reward
        if sim.reward(s) > sim.reward(sBest):
            sBest = s
        
        #Logging?
        jList.append(j)
        rList.append(rAll)


print(sBest)

#should memorize score and print out one with the lowest score


Instructions for updating:
Use `tf.global_variables_initializer` instead.


NameError: name 'sim' is not defined

I am assuming that it optimizes for the highest score possible, therefore, this gets a bit complicated as lower times need a higher score. Could be possible via an inverting function to get the reciprocal of the time as the score

Otherwise I need to figure out where to change it to be minimize instead of maximize. I marked a few places, though it may be easier to ignore it for now and just change the input funciton.